In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
from scipy.stats import chi2_contingency

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

In [5]:
class Analyzer:
    def __init__(self):
        self.A = None
        self.B = None
        self.paired = None
        self.testtype = None
        self.result = None

    def format_prepare(self, data, column=None):
        """
            This method should check for the data to be of right format.
            For example if the data is a DataFrame, it should change the data to Series or numpy array.
        """
        if isinstance(data, np.ndarray):
            if data.ndim > 1:
                return data.flatten()
            return data

        elif isinstance(data, pd.Series):
            return data.values

        elif isinstance(data, pd.DataFrame):
            if column is None:
                if data.shape[1] == 1:
                    return data.iloc[:, 0].values
                else:
                    raise ValueError("DataFrame has multiple columns. Please specify 'column'.")
            else:
                if column not in data.columns:
                    raise ValueError(f"Column '{column}' not found in DataFrame.")
                return data[column].values
        elif isinstance(data, list):
            return np.array(data)

        else:
            raise TypeError("Unsupported data type. Use list, numpy array, pandas Series, or DataFrame.")


    def fit(self, a, b, column_a=None, column_b=None, paired=False):
        """
            This method assigns the data after fixing the format and check for their lengths.
        """
        format_checked_a = self.format_prepare(a, column_a)
        format_checked_b = self.format_prepare(b, column_b)

        if len(format_checked_a) != len(format_checked_b):
            raise ValueError("x and y must have the same length.")
        self.A = format_checked_a
        self.B = format_checked_b
        self.paired = paired

    def summary_check(self):
        return {
            "x_shape": self.A.shape,
            "y_shape": self.B.shape,
            "x_dtype": self.A.dtype,
            "y_dtype": self.B.dtype,
            "paired": self.paired,
            "test type": self.testtype
        }
    def _check_normality(self, data):
        """Check normality using Shapiro-Wilk test (p>0.05 = normal)."""
        stat, p = stats.shapiro(data)
        return p > 0.05

    def analysis_starter(self, alpha=0.05):
        """"
            This method is the analysis starter. it has to check the data for normality and
            also should distinguish between independent and paired data and chooses the test.
        """
        if self.B is None:
            if self._check_normality(self.B):
                self.testtype = "Pearson Correlation"
            else:
                self.testtype = "Spearman Correlation"
            return self.testtype

        if self.paired:
            if self._check_normality(self.A - self.B):
                self.testtype = "Paired t-test"
            else:
                self.testtype = "Wilcoxon Signed-Rank Test"
        else:
            if self._check_normality(self.A) and self._check_normality(self.B):
                var_equal = stats.levene(self.data1, self.data2).pvalue > 0.05
                if var_equal:
                    self.testtype = "Independent t-test (equal var)"
                else:
                    self.testtype = "Welch t-test"
            else:
                self.testtype = "Mann-Whitney U Test"

    def run_test(self):
        """
        Executes the statistical test based on self.testtype.
        Saves the statistic and p-value as attributes.
        """
        if self.testtype is None:
            raise ValueError("Please run start_analysis() first to determine the test type.")

        elif self.testtype == "Pearson Correlation":
            self.result = stats.pearsonr(self.A, self.A)
        elif self.testtype == "Spearman Correlation":
            self.result = stats.spearmanr(self.A, self.A)
        elif self.testtype == "Paired t-test":
            self.result = stats.ttest_rel(self.A, self.B)
        elif self.testtype == "Wilcoxon Signed-Rank Test":
            self.result = stats.wilcoxon(self.A, self.B)

        elif self.testtype == "Independent t-test (equal var)":
            self.result = stats.ttest_ind(self.A, self.B, equal_var=True)
        elif self.testtype == "Welch t-test":
            self.result = stats.ttest_ind(self.A, self.B, equal_var=False)
        elif self.testtype == "Mann-Whitney U Test":
            self.result = stats.mannwhitneyu(self.A, self.B)

        else:
            raise ValueError(f"Unsupported test type: {self.testtype}")

        return self.result


    def interpret(self):
        if self.A is None or self.B is None:
            raise Exception('A and B are empty. You must specify data group a and b with object.fit(a, b).')
        score, p_val = self.result
        if p_val < 0.05:
            print('Rejecting H0 hypothesis')
        else:
            print('Cannot reject H0.')


# Effect size helpers (private)

def _cohen_d_independent(self, a: np.ndarray, b: np.ndarray, equal_var: bool = True) -> float:
    """
    Cohen's d for independent groups.
    If equal_var=False (Welch), use the weighted pooled SD formula (still widely used).
    """
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    na, nb = len(a), len(b)
    ma, mb = np.nanmean(a), np.nanmean(b)
    sa, sb = np.nanstd(a, ddof=1), np.nanstd(b, ddof=1)

    if equal_var:
        # Pooled standard deviation
        sp2 = ((na - 1) * sa**2 + (nb - 1) * sb**2) / (na + nb - 2)
        sp = np.sqrt(sp2)
        if sp == 0:
            return 0.0
        return (ma - mb) / sp
    else:
        # Glass delta variants exist; here use Hedges' g approximation with pooled via weights
        # (common pragmatic choice when equal variance is not assumed)
        wp = ((sa**2) / na + (sb**2) / nb)
        sp = np.sqrt(((sa**2 + sb**2) / 2.0))
        if sp == 0:
            return 0.0
        d = (ma - mb) / sp
        # Optional small-sample correction (Hedges' g)
        df = ( (sa**2/na + sb**2/nb)**2 ) / ( ((sa**2/na)**2)/(na-1) + ((sb**2/nb)**2)/(nb-1) )
        J = 1 - (3/(4*df - 1)) if df > 1 else 1.0
        return float(J * d)


def _cohen_d_paired(self, a: np.ndarray, b: np.ndarray) -> float:
    """
    Cohen's d for paired samples: mean(diff) / sd(diff).
    """
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    d = a - b
    md = np.nanmean(d)
    sd = np.nanstd(d, ddof=1)
    if sd == 0:
        return 0.0
    return md / sd


def _rank_biserial_from_mwu(self, a: np.ndarray, b: np.ndarray, u_stat: float) -> float:
    """
    Rank-biserial correlation derived from Mann–Whitney U:
    r_rb = 1 - (2U) / (n_a * n_b)
    Range: [-1, 1]
    """
    na, nb = len(a), len(b)
    if na == 0 or nb == 0:
        return 0.0
    return 1.0 - (2.0 * u_stat) / (na * nb)


def _rank_biserial_from_wilcoxon(self, a: np.ndarray, b: np.ndarray, w_stat: float) -> float:
    """
    Approximate rank-biserial for Wilcoxon signed-rank:
    r_rb = (W_pos - W_neg) / (W_pos + W_neg)
    We can approximate via: r_rb = 1 - (2 * W_neg) / (W_pos + W_neg)
    SciPy returns the test statistic W = sum of signed ranks positive? Historically it's min(W+, W-).
    Here we compute directly from diffs for clarity.
    """
    a = np.asarray(a, dtype=float); b = np.asarray(b, dtype=float)
    d = a - b
    # Remove zeros (Wilcoxon ignores zeros)
    d = d[d != 0]
    if d.size == 0:
        return 0.0

    # Compute ranks of absolute differences
    ranks = stats.rankdata(np.abs(d))
    W_pos = np.sum(ranks[d > 0])
    W_neg = np.sum(ranks[d < 0])
    denom = (W_pos + W_neg)
    if denom == 0:
        return 0.0
    return (W_pos - W_neg) / denom

    #Part 3A: DataFrame management & safe access
    def set_df(self, df: pd.DataFrame):
        """
        Attach a working DataFrame to the Analyzer instance.
        This DF will be used by univariate/bivariate helper methods.
        """
        if not isinstance(df, pd.DataFrame):
            raise TypeError("set_df expects a pandas DataFrame.")
        # you can set it even if __init__ didn't define it
        self.df = df.copy()
        return self

    def _ensure_df(self):
        """
        Ensure that a DataFrame has been set via set_df().
        """
        if not hasattr(self, "df") or self.df is None:
            raise AttributeError("No DataFrame set. Call set_df(df) first.")

    def _get_series(self, col: str, dropna: bool = True) -> pd.Series:
        """
        Return a single column as a clean Series from self.df.
        - Ensures the column exists
        - Optionally drops NaNs
        """
        self._ensure_df()
        if col not in self.df.columns:
            raise KeyError(f"Column '{col}' not found in the DataFrame.")
        s = self.df[col]
        if dropna:
            s = s.dropna()
        return s

    # Part 2: interpretation & quick report
    def _basic_group_stats(self) -> dict:
        """
        Return simple descriptive stats for A and B (after dropping NaNs).
        """
        if self.A is None or self.B is None:
            raise Exception("A and/or B are empty. Call fit(a, b, ...) first.")
        a = np.asarray(self.A, dtype=float); a = a[~np.isnan(a)]
        b = np.asarray(self.B, dtype=float); b = b[~np.isnan(b)]
        stats_a = {
            "n": int(len(a)),
            "mean": float(np.nanmean(a)) if len(a) else np.nan,
            "std": float(np.nanstd(a, ddof=1)) if len(a) > 1 else np.nan,
            "median": float(np.nanmedian(a)) if len(a) else np.nan,
        }
        stats_b = {
            "n": int(len(b)),
            "mean": float(np.nanmean(b)) if len(b) else np.nan,
            "std": float(np.nanstd(b, ddof=1)) if len(b) > 1 else np.nan,
            "median": float(np.nanmedian(b)) if len(b) else np.nan,
        }
        return {"A": stats_a, "B": stats_b}

    def _effect_size_label(self, value: float) -> str:
        """
        Heuristic label for effect size magnitude.
        Uses Cohen's d style thresholds (also reasonable for rank-biserial abs values).
        """
        if value is None or np.isnan(value):
            return "unknown"
        v = abs(value)
        if v >= 0.8: return "large"
        if v >= 0.5: return "medium"
        if v >= 0.2: return "small"
        return "very small"

    def interpret(self, alpha: float = 0.05, verbose: bool = True):
        """
        Human-friendly interpretation of the latest test result.
        If no result is present, runs the test first.
        Prints a short narrative if verbose=True.
        Returns the result dict augmented with group stats.
        """
        if self.result is None:
            # Ensure test type is selected and run the test
            self.run_test(alpha=alpha)

        res = dict(self.result)  # copy
        groups = self._basic_group_stats()
        res["group_stats"] = groups

        # Build a short message
        test_name = res.get("test_used", "unknown test")
        stat = res.get("statistic", np.nan)
        p = res.get("p_value", np.nan)
        eff = res.get("effect_size", None)
        sig = res.get("significant", False)

        # Pick mean or median difference depending on parametric vs non-parametric
        a_mean, b_mean = groups["A"]["mean"], groups["B"]["mean"]
        a_med, b_med = groups["A"]["median"], groups["B"]["median"]
        if isinstance(test_name, str) and any(k in test_name for k in ["mannwhitney", "wilcoxon"]):
            diff = float(a_med - b_med)
            diff_label = "median(A) - median(B)"
        else:
            diff = float(a_mean - b_mean)
            diff_label = "mean(A) - mean(B)"

        res["difference"] = {"name": diff_label, "value": diff}
        res["effect_label"] = self._effect_size_label(eff)

        if verbose:
            print("=" * 60)
            print(f"Test: {test_name} | alpha={alpha}")
            print(f"Statistic={stat:.4f} | p-value={p:.4g} | significant={sig}")
            if eff is not None:
                print(f"Effect size={eff:.3f} ({res['effect_label']})")
            print(f"{diff_label} = {diff:.4f}")
            print(f"A: n={groups['A']['n']}, mean={groups['A']['mean']:.4f}, median={groups['A']['median']:.4f}, std={groups['A']['std']}")
            print(f"B: n={groups['B']['n']}, mean={groups['B']['mean']:.4f}, median={groups['B']['median']:.4f}, std={groups['B']['std']}")
            print("=" * 60)

        return res

    def quick_report(self, alpha: float = 0.05) -> dict:
        """
        One-call convenience: decide test (if needed), run it, and return a compact dict.
        Does not print anything.
        """
        if self.result is None or self.testtype is None:
            self.analysis_starter(alpha=alpha)
            self.run_test(alpha=alpha)

        groups = self._basic_group_stats()
        out = dict(self.result)
        out["group_stats"] = groups

        # Also include a concise difference metric
        test_name = out.get("test_used", "")
        if isinstance(test_name, str) and any(k in test_name for k in ["mannwhitney", "wilcoxon"]):
            diff_name = "median(A)-median(B)"
            diff_val = float(groups["A"]["median"] - groups["B"]["median"])
        else:
            diff_name = "mean(A)-mean(B)"
            diff_val = float(groups["A"]["mean"] - groups["B"]["mean"])
        out["difference"] = {"name": diff_name, "value": diff_val}
        out["effect_label"] = self._effect_size_label(out.get("effect_size", None))
        return out

    #  Part 3B: Univariate descriptions (use self.df)
    def describe_numeric(self, col: str, bins: int = 10, kde: bool = True, alpha: float = 0.05):
        """
        Univariate summary for a numeric column in self.df:
        - descriptive stats, missing, Shapiro p-value, skew, IQR outliers
        - plots: histogram(+KDE), boxplot, PDF/CDF
        Returns a dict with the computed statistics.
        """
        s = self._get_series(col, dropna=True).astype(float)

        # Basic stats
        desc = s.describe()
        shapiro_p = stats.shapiro(s).pvalue if len(s) >= 3 else np.nan
        is_normal = (shapiro_p > alpha) if not np.isnan(shapiro_p) else False
        skew_val = stats.skew(s)
        q1, q3 = s.quantile(0.25), s.quantile(0.75)
        iqr = q3 - q1
        lower_b, upper_b = q1 - 1.5*iqr, q3 + 1.5*iqr
        n_low = int((s < lower_b).sum()); n_high = int((s > upper_b).sum())
        n_out = n_low + n_high
        pct_out = 100.0 * n_out / len(s) if len(s) else 0.0
        missing = int(self.df[col].isna().sum())

        # --- plots ---
        fig, ax = plt.subplots(2, 2, figsize=(16, 7), gridspec_kw={'height_ratios': (.85, .15)})

        # Histogram (+ KDE)
        sns.histplot(s, kde=kde, ax=ax[0, 0], color='#55A868')
        ax[0, 0].set_title(f'Histogram of {col}')
        ax[0, 0].set_xlabel('')
        ax[0, 0].set_ylabel('Count')

        # Boxplot
        sns.boxplot(x=s, ax=ax[1, 0], color="#5583A8", orient='h')
        label_text = f"Lower outliers: {n_low}\nUpper outliers: {n_high}\nTotal: {n_out} ({pct_out:.1f}%)"
        patch = mpatches.Patch(color='skyblue', label=label_text)
        ax[1, 0].legend(handles=[patch], fontsize=10, loc='upper left', bbox_to_anchor=(1.02, 1))

        # PDF/CDF
        counts, bin_edges = np.histogram(s, bins=bins, density=True)
        pdf = counts / counts.sum() if counts.sum() != 0 else counts
        cdf = np.cumsum(pdf)
        ax[1, 1] = plt.subplot(122)
        plt.plot(bin_edges[1:], pdf, label='PDF')
        plt.plot(bin_edges[1:], cdf, label='CDF')
        plt.legend()
        plt.xticks(rotation=45)

        # Tidy
        ax[0, 0].set_xticklabels([])
        ax[1, 0].set_yticklabels([])
        fig.suptitle(col, fontsize=18)
        plt.tight_layout()
        plt.show()

        info = {
            "count": int(desc.get("count", 0)),
            "mean": float(desc.get("mean", np.nan)),
            "std": float(desc.get("std", np.nan)),
            "min": float(desc.get("min", np.nan)),
            "25%": float(desc.get("25%", np.nan)),
            "50%": float(desc.get("50%", np.nan)),
            "75%": float(desc.get("75%", np.nan)),
            "max": float(desc.get("max", np.nan)),
            "missing": missing,
            "shapiro_p": float(shapiro_p) if not np.isnan(shapiro_p) else None,
            "normal": bool(is_normal),
            "skew": float(skew_val),
            "iqr_lower_bound": float(lower_b),
            "iqr_upper_bound": float(upper_b),
            "outliers_lower": n_low,
            "outliers_upper": n_high,
            "outliers_total_pct": round(pct_out, 2)
        }
        return info

    def describe_categorical(self, col: str, top_n: int = None):
        """
        Univariate summary for a categorical column in self.df:
        - frequency table with percentages
        - bar plot of counts (optionally top_n categories)
        Returns a dict with counts and percents.
        """
        s = self._get_series(col, dropna=False)  # keep NaN to report missing
        counts = s.value_counts(dropna=True)
        if top_n is not None and top_n > 0:
            counts = counts.head(top_n)
        total_non_na = counts.sum()
        perc = (counts / total_non_na * 100.0).round(2) if total_non_na else counts

        # Plot
        plt.figure(figsize=(12, 5))
        sns.barplot(x=counts.index.astype(str), y=counts.values, color="#55A868")
        plt.title(f'Counts of {col}')
        plt.ylabel('Count')
        plt.xlabel(col)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

        info = {
            "missing": int(s.isna().sum()),
            "unique": int(s.nunique(dropna=True)),
            "counts": counts.to_dict(),
            "percents": perc.to_dict()
        }
        return info

    def bin_numeric(self, col: str, bins, labels=None, new_col: str = None,
                    right: bool = True, include_lowest: bool = True):
        """
        Create categorical bins from a numeric column using pd.cut.
        - bins: list/array of bin edges
        - labels: optional labels for each bin
        - new_col: optional name for the new column; defaults to f"{col}_binned"
        Returns the created Series.
        """
        self._ensure_df()
        if new_col is None:
            new_col = f"{col}_binned"
        s = self._get_series(col, dropna=False)
        binned = pd.cut(s, bins=bins, labels=labels, right=right, include_lowest=include_lowest)
        self.df[new_col] = binned
        return self.df[new_col]

    #  Part 3C: Bivariate analyses (use self.df)
    def rel(self, feature: str, target: str, alpha: float = 0.05):
        """
        Dispatcher: choose the appropriate relationship method based on dtypes.
        """
        x = self._get_series(feature, dropna=False)
        y = self._get_series(target, dropna=False)

        # Decide categorical vs numeric
        x_is_cat = self._is_categorical(x)
        y_is_cat = self._is_categorical(y)

        if not x_is_cat and not y_is_cat:
            return self.rel_num_num(feature, target, alpha=alpha)
        elif x_is_cat and y_is_cat:
            return self.rel_cat_cat(feature, target, alpha=alpha)
        else:
            # Ensure (cat, num) order
            if x_is_cat and not y_is_cat:
                return self.rel_cat_num(cat_col=feature, num_col=target, alpha=alpha)
            else:
                return self.rel_cat_num(cat_col=target, num_col=feature, alpha=alpha)

    def rel_num_num(self, col1: str, col2: str, alpha: float = 0.05, gridsize: int = 20):
        """
        Numeric vs Numeric:
        - Normality via Shapiro (if len>=3)
        - Pearson if both normal else Spearman
        - Plots: scatter + hexbin
        Returns dict with test info.
        """
        s1 = self._get_series(col1, dropna=True).astype(float)
        s2 = self._get_series(col2, dropna=True).astype(float)

        # align indices (just in case lengths differ after dropna)
        df_pair = pd.DataFrame({col1: s1, col2: s2}).dropna()
        x = df_pair[col1].values
        y = df_pair[col2].values

        sh1 = stats.shapiro(x).pvalue if len(x) >= 3 else np.nan
        sh2 = stats.shapiro(y).pvalue if len(y) >= 3 else np.nan
        both_normal = (sh1 > alpha if not np.isnan(sh1) else False) and (sh2 > alpha if not np.isnan(sh2) else False)

        if both_normal:
            test_used = "Pearson"
            r, p = stats.pearsonr(x, y)
        else:
            test_used = "Spearman"
            r, p = stats.spearmanr(x, y)

        r2 = r ** 2
        strength = "strong" if abs(r) > 0.7 else ("moderate" if abs(r) > 0.3 else "weak")
        direction = "positive" if r > 0 else "negative"

        # --- plots ---
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle(f'{col1} vs {col2} - {test_used} correlation', fontsize=14)

        # Scatter
        axes[0].scatter(x, y, s=50, alpha=0.7, color='blue', edgecolors='black', linewidth=0.5)
        axes[0].set_title('Scatter')
        axes[0].set_xlabel(col1); axes[0].set_ylabel(col2); axes[0].grid(True, alpha=0.3)

        # Hexbin
        hb = axes[1].hexbin(x, y, gridsize=gridsize, cmap='Blues', mincnt=1)
        axes[1].set_title('Hexbin'); axes[1].set_xlabel(col1); axes[1].set_ylabel(col2)
        cb = fig.colorbar(hb, ax=axes[1]); cb.set_label('count')

        plt.tight_layout(); plt.show()

        return {
            "test_used": test_used,
            "shapiro_p_col1": float(sh1) if not np.isnan(sh1) else None,
            "shapiro_p_col2": float(sh2) if not np.isnan(sh2) else None,
            "correlation": float(r),
            "r_squared": float(r2),
            "p_value": float(p),
            "significant": bool(p < alpha),
            "relationship_strength": strength,
            "relationship_direction": direction,
            "n": int(len(x))
        }

    def rel_cat_num(self, cat_col: str, num_col: str, alpha: float = 0.05):
        """
        Categorical vs Numeric:
        - Shapiro per group; if all normal -> ANOVA; else Welch's ANOVA (if available) else Kruskal-Wallis
        - Plots: box, violin, mean bar, count of categories
        Returns dict with test info + per-group normality.
        """
        s_cat = self._get_series(cat_col, dropna=False)
        s_num = self._get_series(num_col, dropna=True).astype(float)

        # Align and drop NaNs in numeric and category simultaneously
        df_local = pd.DataFrame({cat_col: s_cat, num_col: self.df[num_col]}).dropna()
        # Re-extract arrays
        groups = df_local.groupby(cat_col)[num_col].describe()
        normality = {}
        data_groups = []
        all_normal = True

        for name, grp in df_local.groupby(cat_col):
            vals = grp[num_col].values
            if len(vals) >= 3:
                p_sh = stats.shapiro(vals).pvalue
                is_norm = p_sh > alpha
                normality[name] = {"shapiro_p": float(p_sh), "is_normal": bool(is_norm), "n": int(len(vals))}
                if not is_norm: all_normal = False
            else:
                normality[name] = {"shapiro_p": None, "is_normal": False, "n": int(len(vals))}
                all_normal = False
            data_groups.append(vals)

        # Choose test
        test_used = None
        stat, p_val = np.nan, np.nan
        if len(data_groups) >= 2:
            if all_normal:
                test_used = "ANOVA"
                stat, p_val = stats.f_oneway(*data_groups)
            else:
                # Try Alexander-Govern (Welch's ANOVA in SciPy name) if available
                try:
                    from scipy.stats import alexandergovern
                    result = alexandergovern(*data_groups)
                    stat, p_val = float(result.statistic), float(result.pvalue)
                    test_used = "Welch_ANOVA"
                except Exception:
                    stat, p_val = stats.kruskal(*data_groups)
                    test_used = "Kruskal_Wallis"
        else:
            test_used = "insufficient_groups"

        # --- plots ---
        fig = plt.figure(figsize=(14, 10))

        ax1 = plt.subplot(221)
        sns.boxplot(data=df_local, x=cat_col, y=num_col, ax=ax1)
        ax1.set_title(f'{num_col} by {cat_col}'); plt.xticks(rotation=45)

        ax2 = plt.subplot(222)
        sns.violinplot(data=df_local, x=cat_col, y=num_col, ax=ax2)
        ax2.set_title(f'{num_col} distribution by {cat_col}'); plt.xticks(rotation=45)

        ax3 = plt.subplot(223)
        means = df_local.groupby(cat_col)[num_col].mean().sort_values(ascending=False)
        sns.barplot(x=means.index, y=means.values, ax=ax3)
        ax3.set_title(f'Mean {num_col} by {cat_col}'); plt.xticks(rotation=45)

        ax4 = plt.subplot(224)
        sns.countplot(data=df_local, x=cat_col, ax=ax4)
        ax4.set_title(f'Count of {cat_col}'); plt.xticks(rotation=45)

        plt.tight_layout(); plt.show()

        return {
            "test_used": test_used,
            "all_groups_normal": bool(all_normal),
            "statistic": float(stat) if not np.isnan(stat) else None,
            "p_value": float(p_val) if not np.isnan(p_val) else None,
            "significant": (p_val < alpha) if not np.isnan(p_val) else None,
            "num_categories": int(df_local[cat_col].nunique()),
            "total_observations": int(len(df_local)),
            "group_stats": groups.to_dict(),
            "normality": normality
        }

    def rel_cat_cat(self, col1: str, col2: str, alpha: float = 0.05):
        """
        Categorical vs Categorical:
        - Contingency table + Chi-square test
        - Effect size: Cramér's V
        - Plots: heatmap (counts), stacked 100% bar, normalized heatmap, grouped counts
        Returns dict with chi2, p, dof, Cramér's V.
        """
        s1 = self._get_series(col1, dropna=False).astype("category")
        s2 = self._get_series(col2, dropna=False).astype("category")

        df_local = pd.DataFrame({col1: s1, col2: s2}).dropna()
        table = pd.crosstab(df_local[col1], df_local[col2])
        norm = pd.crosstab(df_local[col1], df_local[col2], normalize='index')
        n = table.values.sum()

        # Chi-square test
        chi2, p_val, dof, expected = stats.chi2_contingency(table)

        # Check expected counts rule-of-thumb
        too_small = (expected < 5).sum()
        expected_ok = (too_small / expected.size) <= 0.2  # no more than 20% < 5

        # Cramér's V
        k = min(table.shape) - 1
        cramers_v = np.sqrt(chi2 / (n * max(k, 1)))

        strength = "strong" if cramers_v > 0.3 else ("moderate" if cramers_v > 0.1 else "weak")

        # --- plots ---
        fig, axs = plt.subplots(2, 2, figsize=(14, 10))

        sns.heatmap(table, annot=True, fmt='d', cbar=False, cmap='YlGnBu',
                    ax=axs[0, 0], linecolor='lightgray', linewidths=0.7)
        axs[0, 0].set_title(f'Contingency: {col1} vs {col2}')

        norm.plot.bar(stacked=True, ax=axs[0, 1])
        axs[0, 1].set_title('Stacked Bar (100%)')
        axs[0, 1].legend(title=col2)
        axs[0, 1].set_xlabel(col1); axs[0, 1].set_ylabel('Proportion')

        sns.heatmap(norm, annot=True, fmt='.2%', cbar=False, cmap='YlGnBu',
                    ax=axs[1, 0], linecolor='lightgray', linewidths=0.7)
        axs[1, 0].set_title('Row-normalized (%)')

        table.plot(kind='bar', ax=axs[1, 1])
        axs[1, 1].set_title('Counts by group'); axs[1, 1].legend(title=col2)
        plt.xticks(rotation=45)

        plt.tight_layout(); plt.show()

        return {
            "chi2_statistic": float(chi2),
            "p_value": float(p_val),
            "degrees_of_freedom": int(dof),
            "cramers_v": float(cramers_v),
            "association_strength": strength,
            "significant_association": bool(p_val < alpha),
            "expected_counts_ok": bool(expected_ok),
            "contingency_table": table.to_dict(),
            "normalized_row": norm.to_dict(),
            "n": int(n)
        }

    #  Part 3D: KPI helpers & plots (use self.df)
    def _get_grouped_data(self, group_cols, kpi: str = "Conversion Rate"):
        """
        Internal helper: group self.df by group_cols and compute KPI.
        Supported KPIs:
          - "Conversion Rate" = conversions / clicks * 100
          - "CPC"             = cost / clicks
        Expected raw columns (if you want auto-compute):
          - 'clicks', 'conversions', 'cost'
        If KPI column already exists in self.df, grouping will aggregate by mean.
        Returns a grouped DataFrame with columns: group_cols + [kpi]
        """
        self._ensure_df()
        if isinstance(group_cols, (str,)):
            group_cols = [group_cols]

        df = self.df.copy()

        # If KPI column missing, try to compute from standard raw columns
        if kpi not in df.columns:
            # Auto-compute if raw columns exist
            if kpi == "Conversion Rate":
                if all(c in df.columns for c in ["conversions", "clicks"]):
                    # Avoid division by zero
                    df["Conversion Rate"] = np.where(df["clicks"] > 0,
                                                     (df["conversions"] / df["clicks"]) * 100.0, np.nan)
                else:
                    raise KeyError("To compute 'Conversion Rate', columns 'conversions' and 'clicks' must exist.")
            elif kpi == "CPC":
                if all(c in df.columns for c in ["cost", "clicks"]):
                    df["CPC"] = np.where(df["clicks"] > 0, df["cost"] / df["clicks"], np.nan)
                else:
                    raise KeyError("To compute 'CPC', columns 'cost' and 'clicks' must exist.")
            else:
                raise ValueError("KPI must be either 'Conversion Rate' or 'CPC'.")

        # Group and aggregate KPI by mean (typical for rates/cost per click after per-row calc)
        grouped = df.groupby(group_cols, dropna=False, as_index=False)[kpi].mean()
        return grouped

    def kpi_heatmap(self, group1: str, group2: str, kpi: str = "Conversion Rate"):
        """
        Draw a heatmap of KPI by two categorical dimensions.
        """
        if kpi not in ["Conversion Rate", "CPC"]:
            raise ValueError('KPI must be "Conversion Rate" or "CPC"')

        tmp = self._get_grouped_data([group1, group2], kpi=kpi)

        heatmap_data = tmp.pivot_table(index=group1, columns=group2, values=kpi)
        heatmap_data = heatmap_data.fillna(0)

        plt.figure(figsize=(18, 8))
        sns.heatmap(
            heatmap_data,
            cmap='YlGnBu',
            annot=True,
            linewidths=0.7,
            linecolor='lightgray'
        )
        plt.title(f'Heatmap of {kpi} by {group1} / {group2}', fontsize=18, weight='bold')
        plt.ylabel(group1); plt.xlabel(group2)
        plt.tight_layout()
        plt.show()

        return heatmap_data  # return matrix for possible reuse

    def kpi_bar(self, group_col: str, kpi: str = "Conversion Rate"):
        """
        Draw a barplot of KPI by a single categorical dimension and annotate bars.
        """
        if kpi not in ["Conversion Rate", "CPC"]:
            raise ValueError('KPI must be "Conversion Rate" or "CPC"')

        tmp = self._get_grouped_data(group_col, kpi=kpi)

        plt.figure(figsize=(12, 7))
        ax = sns.barplot(data=tmp, x=group_col, y=kpi, palette='viridis')
        ax.set_title(f'{kpi} by {group_col}')
        ax.set_xlabel(group_col); ax.set_ylabel(kpi)
        plt.xticks(rotation=45, ha='right')

        # Annotate each bar with a friendly label
        for p in ax.patches:
            val = p.get_height()
            if np.isnan(val):
                label = "NA"
            else:
                label = f'{val:.2f}%' if kpi == "Conversion Rate" else f'{val:.4f}$'
            ax.annotate(label,
                        (p.get_x() + p.get_width()/2., val if not np.isnan(val) else 0),
                        ha='center', va='center', fontsize=11, color='black',
                        xytext=(0, 10), textcoords='offset points')

        plt.tight_layout()
        plt.show()

        return tmp  # return grouped table

    def visual_groups(self, bins: int = 20, kde: bool = True, title: str = None):
        """
        Visual comparison of groups A and B:
        - overlaid histograms (+optional KDE)
        - side-by-side boxplots
        - side-by-side violin plots
        - Q–Q plots vs Normal
        Requirements: self.A and self.B set via fit().
        """
        if self.A is None or self.B is None:
            raise Exception("A and/or B are empty. Call fit(a, b, ...) first.")

        a = np.asarray(self.A, dtype=float)
        b = np.asarray(self.B, dtype=float)
        a = a[~np.isnan(a)]
        b = b[~np.isnan(b)]

        if len(a) == 0 or len(b) == 0:
            raise ValueError("A and/or B contain no valid numeric values after NaN removal.")

        # --- Figure layout ---
        fig = plt.figure(figsize=(16, 10))
        if title is None:
            title = "A vs B — distribution & diagnostics"
        fig.suptitle(title, fontsize=16, y=0.98)

        # 1) Overlaid histograms (+KDE)
        ax1 = plt.subplot(221)
        sns.histplot(a, bins=bins, kde=kde, stat='density', color="#4C78A8", alpha=0.45, ax=ax1, label="A")
        sns.histplot(b, bins=bins, kde=kde, stat='density', color="#F58518", alpha=0.45, ax=ax1, label="B")
        ax1.set_title("Overlaid histograms")
        ax1.set_xlabel(""); ax1.set_ylabel("Density")
        ax1.legend()

        # 2) Boxplots (side-by-side)
        ax2 = plt.subplot(222)
        sns.boxplot(data=[a, b], orient='v', ax=ax2, palette=["#4C78A8", "#F58518"])
        ax2.set_title("Boxplots (A, B)")
        ax2.set_xticklabels(["A", "B"])

        # 3) Violin plots (side-by-side)
        ax3 = plt.subplot(223)
        # Build a compact DataFrame for seaborn violin
        df_tmp = pd.DataFrame({
            "value": np.r_[a, b],
            "group": ["A"] * len(a) + ["B"] * len(b)
        })
        sns.violinplot(data=df_tmp, x="group", y="value", ax=ax3, palette=["#4C78A8", "#F58518"])
        ax3.set_title("Violin plots")

        # 4) Q–Q plots against Normal
        ax4 = plt.subplot(224)
        # For readability, draw QQ for both A and B sequentially
        stats.probplot(a, dist="norm", plot=ax4)
        stats.probplot(b, dist="norm", plot=ax4)
        ax4.set_title("Q–Q plots vs Normal")

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

    def visual_groups_pdf_cdf(self, bins: int = 20):
        """
        Plot PDF/CDF curves for groups A and B in a single axes pair.
        Good for comparing cumulative behavior (stochastic dominance).
        """
        if self.A is None or self.B is None:
            raise Exception("A and/or B are empty. Call fit(a, b, ...) first.")
        a = np.asarray(self.A, dtype=float); a = a[~np.isnan(a)]
        b = np.asarray(self.B, dtype=float); b = b[~np.isnan(b)]

        if len(a) == 0 or len(b) == 0:
            raise ValueError("A and/or B contain no valid numeric values after NaN removal.")

        # Compute PDF/CDF for A
        ca, ba = np.histogram(a, bins=bins, density=True)
        pdf_a = ca / ca.sum() if ca.sum() != 0 else ca
        cdf_a = np.cumsum(pdf_a)

        # Compute PDF/CDF for B
        cb, bb = np.histogram(b, bins=bins, density=True)
        pdf_b = cb / cb.sum() if cb.sum() != 0 else cb
        cdf_b = np.cumsum(pdf_b)

        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        # PDF
        axes[0].plot(ba[1:], pdf_a, label="A - PDF")
        axes[0].plot(bb[1:], pdf_b, label="B - PDF")
        axes[0].set_title("PDF comparison"); axes[0].legend()

        # CDF
        axes[1].plot(ba[1:], cdf_a, label="A - CDF")
        axes[1].plot(bb[1:], cdf_b, label="B - CDF")
        axes[1].set_title("CDF comparison"); axes[1].legend()

        for ax in axes:
            ax.grid(True, alpha=0.25)

        plt.tight_layout()
        plt.show()

    #  Part 5: final wiring & cleanup
    def visual_description(self, bins: int = 20, kde: bool = True, title: str = None):
        """
        Quick visual overview for groups A and B.
        Shows overlaid histograms, box/violin, and QQ plots.
        """
        self.visual_groups(bins=bins, kde=kde, title=title)

    # (optional) keep legacy name but warn; not used anymore
    def __initial_test(self, test):
        """
        DEPRECATED: kept only for backward compatibility.
        Use analysis_starter() + run_test() instead.
        """
        raise DeprecationWarning(
            "Use analysis_starter() to choose the test, then run_test() to execute it."
        )

In [6]:
x1 = [22, 23, 24, 25]
df = df = pd.DataFrame({
    "age": [20, 22, 25, 30],
    "score": [70, 80, 90, 85]
})
x = [12, 15, 14, 10, 13, 15, 16]
y = [2, 2, 2, 2, 2, 6, 8]



In [9]:
stat_ob = Analyzer()
stat_ob.fit(x1, df["score"], paired=True)
stat_ob.analysis_starter()

In [11]:
analyzer_test = Analyzer()
analyzer_test.fit(x, y, paired=False)
analyzer_test.analysis_starter()


In [12]:
stat_ob.summary_check()

{'x_shape': (4,),
 'y_shape': (4,),
 'x_dtype': dtype('int64'),
 'y_dtype': dtype('int64'),
 'paired': True,
 'test type': 'Paired t-test'}

In [13]:
analyzer_test.summary_check()

{'x_shape': (7,),
 'y_shape': (7,),
 'x_dtype': dtype('int64'),
 'y_dtype': dtype('int64'),
 'paired': False,
 'test type': 'Mann-Whitney U Test'}